In [5]:
import csv
import ast
import sys
import os
!conda install --yes --prefix {sys.prefix} pandas
import pandas as pd

Solving environment: ...working... done

# All requested packages already installed.

['C:\\Users\\SDVII\\Documents\\GitHub\\Lyrics-Top-BH100-NLP', 'C:\\Users\\SDVII\\miniconda3\\python37.zip', 'C:\\Users\\SDVII\\miniconda3\\DLLs', 'C:\\Users\\SDVII\\miniconda3\\lib', 'C:\\Users\\SDVII\\miniconda3', '', 'C:\\Users\\SDVII\\miniconda3\\lib\\site-packages', 'C:\\Users\\SDVII\\miniconda3\\lib\\site-packages\\win32', 'C:\\Users\\SDVII\\miniconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\SDVII\\miniconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\SDVII\\miniconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\SDVII\\.ipython', 'C:\\Program Files\\JetBrains\\PyCharm Professional Edition with Anaconda plugin 2019.1.1\\helpers\\pydev', 'C:\\Program Files\\JetBrains\\PyCharm Professional Edition with Anaconda plugin 2019.1.1\\helpers-pro\\jupyter_debug', 'C:\\Users\\SDVII\\Anaconda3', 'C:\\Users\\SDVII\\Anaconda3', 'C:\\Users\\SDVII\\Anaconda3\\Lib\\site-packages']


In [ ]:
"""Run this section only if anaconda in having trouble finding the packages"""
user_path = "C:\\Users\\SDVII"
sys.path.append(user_path+"\\Anaconda3\\Lib\\site-packages")

In [67]:
"""Generate RAW .txt files for all lyrics"""
import re

def lyrics_decoder(lyrics):
    return ast.literal_eval(lyrics).decode('utf8')

with open('lyrics/bh100.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    
    for song in reader:
        song_rank = song[0]
        song_name = re.sub(r"[^a-zA-Z0-9 ']", '', song[1])
        song_artist = song[2]
        song_year = song[3]
        song_lyrics = re.sub(r"[^a-zA-Z0-9 ']", ' ', lyrics_decoder(song[4]))
        
        if not os.path.exists('lyrics/raws/'+song_year):
            os.mkdir('lyrics/raws/'+song_year)
        
        with open('lyrics/raws/'+song_year+'/'+song_name+'.txt', 'w', newline="") as txt_file:
            txt_file.write(song_lyrics)

In [99]:
import pke
import string
from nltk.corpus import stopwords

def keyphrases_formater(keyphrases):
    result = ""
    for index,keyphrase in enumerate(keyphrases):
        result += keyphrase[0].split(' ', 1)[0]
        if index<len(keyphrases)-1:
           result += ";"
            
    return result

with open('results/keywords.csv', 'w', newline="") as csvfile:
    filewriter = csv.writer(
        csvfile,
        delimiter=',',
        quotechar='|',
        quoting=csv.QUOTE_MINIMAL
    )
    
    filewriter.writerow(['Rank', 'Song', 'Artist', 'Year','Topic Rank','Single Rank','Position Rank','Multipartite Rank'])

    with open('lyrics/bh100.csv', 'r') as bh100:
        bh_reader = csv.reader(bh100, delimiter=',')
        next(bh_reader)
        
        for song in bh_reader:
            song_rank = song[0]
            song_name = song[1]
            song_file = re.sub(r"[^a-zA-Z0-9 ']", '', song[1])
            song_artist = song[2]
            song_year = song[3]
                
            """Extractors' variables"""
            pos = {'NOUN', 'PROPN', 'ADJ'}
            grammar = "NP: {<ADJ>*<NOUN|PROPN>+}"
            stoplist = list(string.punctuation)
            stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
            stoplist += stopwords.words('english')
            
            """Topic Rank"""
            tr_extractor = pke.unsupervised.TopicRank()
            tr_extractor.load_document(input='lyrics/raws/'+song_year+'/'+song_file+'.txt', language='en')
            tr_extractor.candidate_selection(pos=pos, stoplist=stoplist)
            tr_extractor.candidate_weighting(threshold=0.74, method='average')
            tr_keyphrases = keyphrases_formater(tr_extractor.get_n_best(n=20))
            
            """Single Rank"""
            sr_extractor = pke.unsupervised.SingleRank()
            sr_extractor.load_document(input='lyrics/raws/'+song_year+'/'+song_file+'.txt',language='en',normalization=None)
            sr_extractor.candidate_selection(pos=pos)
            sr_extractor.candidate_weighting(window=10, pos=pos)
            sr_keyphrases = keyphrases_formater(sr_extractor.get_n_best(n=20))
   
            """Position Rank"""
            pr_extractor = pke.unsupervised.PositionRank()
            pr_extractor.load_document(input='lyrics/raws/'+song_year+'/'+song_file+'.txt',language='en',normalization=None)
            pr_extractor.candidate_selection(grammar=grammar,maximum_word_number=3)
            pr_extractor.candidate_weighting(window=10,pos=pos)
            pr_keyphrases = keyphrases_formater(pr_extractor.get_n_best(n=20))
            
            """Multipartite Rank"""
            mr_extractor = pke.unsupervised.MultipartiteRank()
            mr_extractor.load_document(input='lyrics/raws/'+song_year+'/'+song_file+'.txt')
            mr_extractor.candidate_selection(pos=pos, stoplist=stoplist)
            mr_extractor.candidate_weighting(alpha=1.1,threshold=0.74,method='average')
            mr_keyphrases = keyphrases_formater(mr_extractor.get_n_best(n=20))
            
            filewriter.writerow([song_rank, song_name, song_artist, song_year,tr_keyphrases,sr_keyphrases,pr_keyphrases,mr_keyphrases])
